In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

logger = Logging.setup_logging()


class CircuitSimulation:
    def __init__(self, circuit, steptime, switchingtime, finaltime):
        self.circuit = circuit
        self.steptime = steptime
        self.switchingtime = switchingtime
        self.finaltime = finaltime

    def simulate_transient_response(self):
        simulator = self.circuit.simulator(temperature=25, nominal_temperature=25)
        ic = 0@u_V
        simulator.initial_condition(output=ic)
        analysis = simulator.transient(step_time=self.steptime, end_time=self.finaltime)
        return analysis


class PlotGenerator:
    def __init__(self, analysis):
        self.analysis = analysis

    def plot_switch_activations(self):
        plt.title('Switch activations')
        plt.ylabel('Voltage [V]')
        plt.grid()
        plt.plot(self.analysis['posa'])
        plt.plot(self.analysis['posb'])
        plt.legend(('position a', 'position b'), loc=(.05, .1))
        plt.yticks([-1, 0, 1])
        plt.show()

    def plot_voltage_across_capacitor(self):
        plt.title('Voltage across Capacitor')
        plt.xlabel('Time [s]')
        plt.ylabel('Voltage [V]')
        plt.grid()
        plt.plot(self.analysis['output'], color='black')
        plt.plot(self.analysis['2'], color='magenta')
        plt.legend(('Output', 'v_R4'), loc=(.05, .1))
        plt.yticks([0, 100-(100/math.exp(1)), 100/math.exp(1), 100])
        plt.show()

    def plot_currents(self):
        plt.title('Currents')
        plt.xlabel('Time [s]')
        plt.ylabel('Current [A]')
        plt.grid()
        plt.plot((self.analysis['1']-self.analysis['a'])/self.circuit.R1.resistance, color='black')
        plt.plot(self.analysis['2']/self.circuit.R4.resistance, color='magenta')
        plt.plot((self.analysis['b']-self.analysis['2'])/self.circuit.R2.resistance, color='blue')
        plt.legend(('i_R1', 'i_R4', 'i_R2'), loc=(.05, .1))
        plt.show()


def main():
    steptime = 1@u_us
    switchingtime = 20@u_ms
    finaltime = 80@u_ms

    circuit_response = Circuit('Step Response of RC Circuit')
    # Defina o circuito aqui

    simulation = CircuitSimulation(circuit_response, steptime, switchingtime, finaltime)
    analysis = simulation.simulate_transient_response()

    plot_generator = PlotGenerator(analysis)
    plot_generator.plot_switch_activations()
    plot_generator.plot_voltage_across_capacitor()
    plot_generator.plot_currents()


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'PySpice'